In [1]:
import sys
sys.path.append("/home/apbarret/src/SunlightUnderSeaIce/sunderseaice")  # Allow import of modules for sunderseaice
sys.path.append("/home/apbarret/src/nsidc_projections/nsidc_projections")

import os
import datetime as dt
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LinearRing, Polygon
from shapely.geometry.polygon import orient
from pyproj import Transformer

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

import h5py
import xarray as xr

import download.nsidc_download as cmr
from download.icesat2_search import read_polarstern_track, select_midday, POLARSTERN_TRACK_FILE
from grid import SSMI_PolarStereoNorth25km 

DATAPATH = '/home/apbarret/src/SunlightUnderSeaIce/data'

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    map_proj = SSMI_PolarStereoNorth25km.to_cartopy()
map_extent = [-2349878.8355433852, 2349878.8355433857, -2349878.8355433852, 2349878.8355433857]

In [13]:
product = 'ATL03'
version = '004'

search_date = dt.datetime(2020, 6, 19, 12, 0, 0)
start_date, end_date = search_date - dt.timedelta(hours=12), search_date + dt.timedelta(hours=12)
search_polygon = '10.078302880815441,81.817809200106,10.512323997285574,81.89034790835252,10.636753149555467,81.9838464717769,10.408064655537508,82.07352694485158,9.878747549516635,82.13485973955459,9.1925222962977,82.15062045169547,8.544241862012756,82.11632240869798,8.11617137324634,82.04169749713458,8.020321719821991,81.94751873980789,8.27166097082281,81.8592435441352,8.795052842963516,81.79999741914631,9.452676702124208,81.78491763339973,10.078302880815441,81.817809200106'

results = cmr.cmr_search(
    product, 
    version, 
    start_date.isoformat(),  # CMR API expects iso format date strings
    end_date.isoformat(), 
    polygon=search_polygon,
    verbose=True,
    return_search_page=True,
    )
print(results)

Querying for data:
	https://cmr.earthdata.nasa.gov/search/granules.json?provider=NSIDC_ECS&sort_key[]=start_date&sort_key[]=producer_granule_id&scroll=true&page_size=2000&short_name=ATL03&version=004&version=04&version=4&temporal[]=2020-06-19T00:00:00,2020-06-20T00:00:00&polygon[]=10.078302880815441,81.817809200106,10.512323997285574,81.89034790835252,10.636753149555467,81.9838464717769,10.408064655537508,82.07352694485158,9.878747549516635,82.13485973955459,9.1925222962977,82.15062045169547,8.544241862012756,82.11632240869798,8.11617137324634,82.04169749713458,8.020321719821991,81.94751873980789,8.27166097082281,81.8592435441352,8.795052842963516,81.79999741914631,9.452676702124208,81.78491763339973,10.078302880815441,81.817809200106

Found 1 matches.
{'feed': {'updated': '2021-05-26T22:33:48.776Z', 'id': 'https://cmr.earthdata.nasa.gov:443/search/granules.json?provider=NSIDC_ECS&sort_key[]=start_date&sort_key[]=producer_granule_id&scroll=true&page_size=2000&short_name=ATL03&version=0

In [15]:
entries = [e['links']
               for e in results['feed']['entry']
               if 'links' in e]
print(entries)

[]
